# Computing the area of a frontier's 2D cross section
Given a Pareto-optimal frontiers f1, for each 2D cross section of the frontier, this program computes the area bounded by the solutions in f1 that are non-dominated in the cross section. Assuming, wlog, that all objectives are MAX, the larger the area, the less conflict.

In [36]:
import pandas as pd
import numpy as np
import itertools
import time

### Preparation for volume computation algorithm

In [45]:
# Get the solutions that define the pareto frontier.
# It is assumed that these frontiers do not contain any dominated solutions
# (that is, that the solutions are truly Pareto optimal)

sols = pd.read_csv("../solutionSets/3d/ClimateChange_E85/climateChange_EfficientSolutions_E85.csv")

In [46]:
# Hard-coded manipulations to tidy-up sols

# set index, remove unwanted columns
sols = sols.set_index("1-" + sols["SolutionIndex"].astype(str))
sols = sols.drop(["SolutionIndex","Frontier"],axis=1)

In [47]:
# Get list of objectives from the dataframe

objs = sols.columns.tolist()

In [48]:
# Create empty dictionary to hold the objectives and the senses of each
# This process is hard-coded and requires some knowledge about the model
# that resulted in the dataframe

objSenses = {}

# Hard-coded manipulations to properly set objSenses (max/min)
# 1 for max, and 0 for min

for obj in objs:
    objSenses[obj] = 0
objSenses["Northern Spotted Owl Habitat (ha)"] = 1
#objSenses["PER"] = 0
#objSenses

In [49]:
def maxAllAndNorm(origf,objSenses):
    
    f = origf
    
    # get bounds on the objectives' values
    objBounds = {}
    for obj,sense in objSenses.items():
        objBounds[obj] = [f[obj].min(),f[obj].max()]
    # get each objective's ideal value
    idealObjVals = {}
    for obj,bounds in objBounds.items():
        idealObjVals[obj] = bounds[objSenses[obj]] # 0th entry of bounds is min, 1st is max
    # get each objective's worst value
    worstObjVals = {}
    for obj,bounds in objBounds.items():
        worstObjVals[obj] = bounds[not(objSenses[obj])]
        
    # Normalize the objective spaces by converting each value to the relative achievement along its axis:

    #   distance from the worst case value
    # ---------------------------------------
    # total distance spanned by the objective

    # Normalized objective space is in [0,1]^N where N = # of objectives


    for obj,bounds in objBounds.items():
        f[obj] = f[obj].apply(
            lambda x: \
            abs(x - worstObjVals[obj]) / \
            (bounds[1]-bounds[0]))
    
    return f

In [50]:
# Convert and normalize the data

sols = maxAllAndNorm(sols,objSenses)

In [51]:
def removeDominatedAndWeaklyDominatedSolutions(f):
    '''
    Given a dataframe of solutions, returns a dataframe with only
    nondominated solutions.
    
    It is assumed that each column is an objective and that all objectives
    aim to be maximized.
    '''
    newf = f

    objectives = f.columns.tolist()
    
    for idx,row in f.iterrows(): # searching for a solution that dominates this one
        for idx2,row2 in f.iterrows(): # scanning over all other solutions
            if np.all((row[objectives] <= row2[objectives])) and idx2 != idx: # if we find a solution that dominates idx,
                newf = newf.drop(idx) # then we remove that index from the to-be-returned dataframe
                break # and stop searching for a dominating solution
            
    return newf

### Algorithm to compute volume of frontier

In [52]:
# Get the non-dominated sets for each 2D cross-section

for objPair in itertools.combinations(objs,2):
    op = list(objPair)
    # prepare the non-dominated set for the area computation
    # by removing the dominated points and sorting it in descending order according to the first objective
    csf = removeDominatedAndWeaklyDominatedSolutions(sols[op]).sort_values(by=[op[0]],ascending=False)
    csvol = np.prod(csf.ix[0])
    for i in np.arange(1,len(csf)):
        csvol += (csf.ix[i][op[1]]-csf.ix[i-1][op[1]])*csf.ix[i][op[0]]
    print (str(op) + ": " + str(csvol))
    print ("------------------")

['Fire Hazard Increase', 'Northern Spotted Owl Habitat (ha)']: 0.990132672219
------------------
['Fire Hazard Increase', 'Peak Sediment Delivery (t)']: 0.848770430413
------------------
['Northern Spotted Owl Habitat (ha)', 'Peak Sediment Delivery (t)']: 0.999999990663
------------------
